# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Введение в обработку текста на естественном языке"
* https://www.nltk.org/api/nltk.metrics.distance.html
* https://pymorphy2.readthedocs.io/en/stable/user/guide.html
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
# !pip install pymorphy2

In [2]:
import nltk
# nltk.download('punkt')

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. При помощи расстояния Левенштейна иправьте опечатку в слове "велечайшим".

In [3]:
from nltk import edit_distance

In [4]:
with open("litw-win.txt", "r", encoding="windows-1251") as fp:
    words = [line.split()[-1] for line in fp]

In [5]:
words[10100:10105]

['науке', 'некто', 'немка', 'неско', 'ножка']

In [6]:
edit_distance("Вы поступили в Ранхигс", "Вы поступили в Финашку")

6

In [7]:
word = "перекренсток"
min(words, key=lambda w: edit_distance(word, w))

'перекресток'

2. Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.

In [8]:
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
import pymorphy2

In [9]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''

In [10]:
words = word_tokenize(text)
words[:5]

['Разбейте', 'текст', 'из', 'формулировки', 'второго']

In [11]:
stemmer = SnowballStemmer("russian")
{w: stemmer.stem(w) for w in words}

{'Разбейте': 'разб',
 'текст': 'текст',
 'из': 'из',
 'формулировки': 'формулировк',
 'второго': 'втор',
 'задания': 'задан',
 'на': 'на',
 'слова': 'слов',
 '.': '.',
 'Проведите': 'провед',
 'стемминг': 'стемминг',
 'и': 'и',
 'лемматизацию': 'лемматизац',
 'слов': 'слов'}

In [12]:
morph = pymorphy2.MorphAnalyzer()

w = words[0]
res = morph.parse(w)
print(res)
opt1 = res[0]

[Parse(word='разбейте', tag=OpencorporaTag('VERB,perf,tran plur,impr,excl'), normal_form='разбить', score=1.0, methods_stack=((DictionaryAnalyzer(), 'разбейте', 646, 14),))]


In [13]:
print(opt1.normalized, end='\n\n')
print(opt1.normalized.word)

Parse(word='разбить', tag=OpencorporaTag('INFN,perf,tran'), normal_form='разбить', score=1.0, methods_stack=((DictionaryAnalyzer(), 'разбить', 646, 0),))

разбить


In [14]:
morph = pymorphy2.MorphAnalyzer()
dct = {w: morph.parse(w)[0].normalized.word for w in words}
# {i: list(dct.values()).count(i) for i in list(dct.values())}
dct

{'Разбейте': 'разбить',
 'текст': 'текст',
 'из': 'из',
 'формулировки': 'формулировка',
 'второго': 'второй',
 'задания': 'задание',
 'на': 'на',
 'слова': 'слово',
 '.': '.',
 'Проведите': 'провести',
 'стемминг': 'стемминг',
 'и': 'и',
 'лемматизацию': 'лемматизация',
 'слов': 'слово'}

3. Преобразуйте предложения из формулировки задания 2 в векторы при помощи `CountVectorizer`. Выведите на экран словарь обученного токенизатора.

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import sent_tokenize

In [16]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''

In [17]:
sents = sent_tokenize(text)
cv = CountVectorizer().fit(sents)
sents_cv = cv.transform(sents).toarray()
sents_cv

array([[1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0]], dtype=int64)

In [18]:
cv.vocabulary_

{'разбейте': 6,
 'текст': 10,
 'из': 2,
 'формулировки': 11,
 'второго': 0,
 'задания': 1,
 'на': 4,
 'слова': 8,
 'проведите': 5,
 'стемминг': 9,
 'лемматизацию': 3,
 'слов': 7}

## Лабораторная работа 7

In [19]:
import pandas as pd
import re
import numpy as np

1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [20]:
recipes = pd.read_csv('ru_recipes_sample.csv', encoding='utf-8')
recipes.head(3)

,url,name,ingredients,description
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",Этот коктейль готовлю из замороженной клубники...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",Быстро и вкусно.
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...","Сытный, овощной салатик, пальчики оближете."


In [21]:
patt = re.compile(r"[^\s\d\А-Яа-я]", re.I)  
recipes.description = recipes.description.apply(lambda x: str.lower(re.sub(patt, '', x).replace('\r', '').replace('\n', '')))
recipes.description

0       этот коктейль готовлю из замороженной клубники...
1                                         быстро и вкусно
2                сытный овощной салатик пальчики оближете
3       картофельное пюре и куриные котлеты  вкусная к...
4       вишневая наливка имеет яркий вишневый вкус кот...
                              ...                        
3462    для тех кто любит чечевицу вам сюда очень вкус...
3463    баклажановые фантазии продолжаются предлагаю в...
3464    мое любимое блюдо лазанья но кушать только фар...
3465    прошлым летом варила варенье из одуванчиков по...
3466     и три корочки хлеба  сделал заказ буратино в ...
Name: description, Length: 3467, dtype: object

### Расстояние редактирования

**2\.** Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [22]:
words = list(set(np.concatenate(recipes.description.apply(lambda row: nltk.word_tokenize(row)).values)))
len(words)

16767

In [23]:
words_for_lev = np.random.choice(words,(5,2))
for pair in words_for_lev:
    print(f'd({pair[0]},{pair[1]}) = {edit_distance(pair[0],pair[1])}')

d(обусловлена,создания) = 9
d(зеленого,зубок) = 6
d(женщин,рулет) = 6
d(необходимость,написал) = 10
d(примитивен,начинающие) = 9


**3\.** Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$. 

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [24]:
def is_plagiarism(s1: str, s2: str) -> bool:
    s1, s2 = nltk.word_tokenize(s1), nltk.word_tokenize(s2)
    L = max(len(s1), len(s2))
    P = 0
    
    for w1 in s1:
        for w2 in s2:
            if edit_distance(w1, w2) < 2:
                P += 1
                break

    if P/L > 0.8:
        return True
    else:
        return False

In [25]:
recipes['id'] = recipes.url.apply(lambda id: int(re.compile(r"\d+", re.I).findall(id)[0]))

In [26]:
s1 = recipes.iloc[np.where(recipes.id == 135488)].description.values[0]
s2 = recipes.iloc[np.where(recipes.id == 851934)].description.values[0]
print(f'Рецепт №1: {s1}', f'Рецепт №2: {s2}', f'Наличие плагиата: {is_plagiarism(s1, s2)}', sep='\n\n')

Рецепт №1: прекрасной закуской к крепким напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком который мы спрячем в орешках из бородинского хлеба положив в середину маслину закуска получается необычной оригинальной и ценится особенно мужчинами

Рецепт №2: замечательной закуской к напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком куда мы добавим орешков из бородинского хлеба положив в середину маслины закуска получается крайне необычной оригинальной и ценится особенно мужчинами

Наличие плагиата: True


### Стемминг, лемматизация

**4\.** На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

**Стемминг**
(слова без наличия условных форм)

In [27]:
stemmer = SnowballStemmer("russian")
stemmed = [stemmer.stem(word) for word in words]

**Лемматизация**
(слова в их нормальной форме)

In [28]:
morph = pymorphy2.MorphAnalyzer()
lemmed = [morph.parse(word)[0].normalized.word for word in words]

In [29]:
stem_lem_df = pd.DataFrame(data=[stemmed, lemmed],
                           columns=words,
                           index=['stemmed_word','normalized_word']).T
stem_lem_df.index.name = 'word'

stem_lem_df

,stemmed_word,normalized_word
word,,
мом,мом,мом
деремся,дер,драться
опарой,опар,опара
сделали,сдела,сделать
кулинарными,кулинарн,кулинарный
...,...,...
верит,вер,верить
отец,отец,отец
оксане,оксан,оксана


In [30]:
stem_lem_df.iloc[10:11] 
# Слово "награды" – существительное женского рода множественного числа.
# С помощью процедуры стемминга мы "отрезаем" наше слово с конца на некоторое количество символов 
# – в данном случае слово осталось без окончания
# С помощью процедуры лемматизации мы получаем слово в его начальной форме – в д.с. в форме женского рода единственного числа

,stemmed_word,normalized_word
word,,
награды,наград,награда


**5\.** Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [31]:
from collections import Counter

In [32]:
from nltk.corpus import stopwords
# nltk.download('stopwords')

In [33]:
rus_stopwords = stopwords.words("russian")
words = list(np.concatenate(recipes.description.apply(lambda row: nltk.word_tokenize(row)).values)) # не убираем повторы

In [34]:
stopwords_quantity = dict()
for word in rus_stopwords:
    if word in Counter(words).keys():
        stopwords_quantity[word] = Counter(words)[word]/len(words)

In [35]:
pd.DataFrame.from_dict(stopwords_quantity, orient='index', columns=['Частота использования'])

,Частота использования
и,0.049205
в,0.025047
во,0.000995
не,0.014831
что,0.007406
...,...
более,0.000761
всегда,0.001376
конечно,0.000790
всю,0.000059


In [67]:
sum(stopwords_quantity.values()) # Общая доля стоп-слов среди всех слов в описаниях

0.3224248455931854

In [36]:
sents_wo_stop = list()
for desc in recipes.description.apply(lambda row: nltk.word_tokenize(row)).values:
    sents_wo_stop.append((' ').join([word for word in desc if word not in rus_stopwords]))

recipes['description_no_stopwords'] = sents_wo_stop
recipes[['description','description_no_stopwords']].head()

,description,description_no_stopwords
0,этот коктейль готовлю из замороженной клубники...,коктейль готовлю замороженной клубники клубник...
1,быстро и вкусно,быстро вкусно
2,сытный овощной салатик пальчики оближете,сытный овощной салатик пальчики оближете
3,картофельное пюре и куриные котлеты вкусная к...,картофельное пюре куриные котлеты вкусная клас...
4,вишневая наливка имеет яркий вишневый вкус кот...,вишневая наливка имеет яркий вишневый вкус кот...


In [37]:
words_wo_stop = list(np.concatenate(recipes.description_no_stopwords.apply(lambda row: nltk.word_tokenize(row)).values))

In [38]:
print('Со stop-словами', *Counter(words).most_common(10),'\nБез stop-слов', *Counter(words_wo_stop).most_common(10),sep='\n')

Со stop-словами
('и', 5043)
('в', 2567)
('с', 1932)
('на', 1642)
('очень', 1594)
('не', 1520)
('из', 1005)
('я', 972)
('а', 850)
('рецепт', 843)

Без stop-слов
('очень', 1594)
('рецепт', 843)
('это', 728)
('блюдо', 521)
('вкусный', 459)
('просто', 434)
('вкусно', 366)
('приготовить', 342)
('вкус', 319)
('салат', 312)


### Векторное представление текста

**6\.** Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [53]:
oladyi = recipes.iloc[np.where(recipes.name.str.extract(r"(.*(?:оладьи).*)", flags=re.IGNORECASE).isnull()==False)[0]].sample(5)
oladyi

,url,name,ingredients,description,id,description_no_stopwords
464,https://www.povarenok.ru/recipes/show/133050/,Яблочные оладьи,"{'Яблоко': '2 шт', 'Яйцо куриное': '1 шт', 'Му...",очень простое бюджетное блюдо особенно понрави...,133050,очень простое бюджетное блюдо особенно понрави...
1745,https://www.povarenok.ru/recipes/show/81631/,Воздушные оладьи,"{'Фарш мясной': '500 г', 'Лук репчатый': '1 шт...",это отличное блюдо для завтрака обеда ужина т...,81631,это отличное блюдо завтрака обеда ужина тесто ...
1882,https://www.povarenok.ru/recipes/show/30148/,Оладьи от Эльзары,"{'Мука пшеничная': '0,5 кг', 'Яйцо куриное': '...",моя подруга эльзара готовит пальчики оближешь...,30148,подруга эльзара готовит пальчики оближешь выпр...
2830,https://www.povarenok.ru/recipes/show/161557/,Оладьи из печени,"{'Печень говяжья': '1000 г', 'Лук репчатый': '...",31 декабря рабочий день времени на подготовку...,161557,31 декабря рабочий день времени подготовку пра...
1139,https://www.povarenok.ru/recipes/show/6443/,Куриные оладьи,"{'Грудка куриная': None, 'Лук репчатый': '2 шт...",на скорую руку за фото к рецепту спасибо людми...,6443,скорую руку фото рецепту спасибо людмиле сурик


### Сначала я использовал код ниже для столбцов с изначальным описанием, затем – для столбцов с описанием без стоп-слов. Второй результат оказался лучше (0.93 → 0.87).

In [54]:
tv = TfidfVectorizer().fit(oladyi.description_no_stopwords)
sents_tv = tv.transform(oladyi.description_no_stopwords).toarray()

In [55]:
pd.DataFrame(sents_tv, 
             columns=sorted(tv.vocabulary_.keys()),
             index=range(1,6))

,31,блендер,блины,блюдо,быстрее,быстро,бюджетное,взрослым,вкусная,вкусные,...,тесто,тефлоновой,ужина,упростился,фото,хозяек,чашу,эльзара,это,яблочками
1,0.000000,0.000000,0.000000,0.280011,0.000000,0.000000,0.347067,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.164504,0.132721,0.164504,0.164504,0.000000,0.164504,0.000000,0.164504,...,0.164504,0.164504,0.164504,0.000000,0.000000,0.164504,0.000000,0.000000,0.164504,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.285112,0.000000,0.285112
4,0.153121,0.153121,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153121,0.000000,...,0.000000,0.000000,0.000000,0.153121,0.000000,0.000000,0.153121,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.377964,0.000000,0.000000,0.000000,0.000000,0.000000


**7\.** Вычислите близость между каждой парой рецептов, выбранных в задании 6, используя косинусное расстояние (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

$${\displaystyle {\text{косинусное сходство}}=S_{C}(A,B)={\mathbf {A} \cdot \mathbf {B} \over \|\mathbf {A} \|\|\mathbf {B} \|}={\frac {\sum \limits _{i=1} ^{n}{A_{i}B_{i}}}{{\sqrt {\sum \limits _{i=1} ^{n}{A_{i}^{2}}}}{\sqrt {\sum \limits _{i=1}^{n}{B_{i}^{2}}}}}}}$$

$${\displaystyle \scriptsize\text{* чем меньше угол между двумя векторами, тем больше сходство двух векторов}}$$

In [56]:
from scipy.spatial.distance import cosine

In [57]:
cosdist = np.reshape([cosine(A,B) for A in sents_tv for B in sents_tv],(5,5))
cosdist_df = pd.DataFrame(cosdist,columns=oladyi.name.values, index=oladyi.name.values).round(3)
cosdist_df

,Яблочные оладьи,Воздушные оладьи,Оладьи от Эльзары,Оладьи из печени,Куриные оладьи
Яблочные оладьи,0.000,0.926,1.000,1.000,1.0
Воздушные оладьи,0.926,0.000,0.969,1.000,1.0
Оладьи от Эльзары,1.000,0.969,0.000,0.972,1.0
Оладьи из печени,1.000,1.000,0.972,0.000,1.0
Куриные оладьи,1.000,1.000,1.000,1.000,0.0


**8\.** Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [58]:
def find_closest(sim_df: pd.DataFrame) -> tuple:
    return sim_df.stack().iloc[(np.where(np.concatenate(sim_df.values)!=0)[0])].idxmin()

In [59]:
find_closest(cosdist_df) 
# Среди пяти выбранных рецептов значения косинусного получились довольно большими и близкими друг к другу.
# Наиболее похожими друг на друга являются рецепты, выведенные ниже. Эти рецпты между собой имеют одно общее слово. 
# Этого оказалось достаточно, чтобы быть по своему содержанию ближе друг к другу, чем остальным рецептам.

('Яблочные оладьи', 'Воздушные оладьи')

In [62]:
recipes.iloc[np.where(recipes.name == find_closest(cosdist_df)[0])].description_no_stopwords.values[0],\
recipes.iloc[np.where(recipes.name == find_closest(cosdist_df)[1])].description_no_stopwords.values[0]

('очень простое бюджетное блюдо особенно понравится кому молочные продукты',
 'это отличное блюдо завтрака обеда ужина тесто замешивается очень быстро пекутся оладьи быстрее зачастую блины неопытных хозяек рвутся ломаются время переворачивания маленькими оладушками такого случается пышные необычайно вкусные придутся душе детям взрослым тефлоновой сковороде такие оладушки пожарить масла')

In [63]:
set(nltk.word_tokenize(recipes.iloc[np.where(recipes.name==find_closest(cosdist_df)[0])].description_no_stopwords.values[0]))\
& set(nltk.word_tokenize(recipes.iloc[np.where(recipes.name==find_closest(cosdist_df)[1])].description_no_stopwords.values[0]))

{'блюдо', 'очень'}